In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import cmath
import math
import time

In [63]:
S0 = 100
K = 100
T = 1
r = 0.05
q = 0.01
k = np.log(K)

#GBM Parameters
sigmaGBM = 0.30

#Variance Gamma Parameters
sigmaVG = 0.30
thetaVG = 0.02
nu = 0.1

#Heston parameters
V0 = 0.04
thetaH = 2.0
sigmaH = 0.05
kappa = 0.3
rho = -0.7



In [64]:
class GeometricBrownianMotion:

    sigma_  = 0.0

    def __init__(self,sigma):
        self.sigma_ = sigma

    def testInit(self):
        print("This is my GBM")
    
    def CharacteristicFunc(self,u,S0,T,r,q):
        phi = np.exp(1j*(np.log(S0)+(r-q-self.sigma_**2/2)*T)*u - ((self.sigma_*np.sqrt(T))*u)**2/2)
        return phi

In [65]:
GBM = GeometricBrownianMotion(sigmaGBM)

In [66]:
GBM.testInit()

This is my GBM


In [67]:
class VarianceGamma:

    sigma_ = 0.0
    theta_ = 0.0
    nu_ = 0.0

    def __init__(self,sigma,nu,theta):
        self.sigma_ = sigma
        self.theta_ = theta
        self.nu_ = nu
       

    def testInit(self):
        print("This is my VG")
    
    def CharacteristicFunc(self,u,S0,T,r,q):
            
            if (self.nu_ == 0):
                #mu = np.log(S0) + (r-q - self.theta_ -0.5*self.sigma_**2)*T
                phi  = np.exp(1j*u*(np.log(S0) + (r-q - self.theta_ -0.5*self.sigma_**2)*T)) * np.exp((1j*self.theta_*u-0.5*self.sigma_**2*u**2)*T)
            else:
                mu  = np.log(S0) + (r-q + np.log(1-self.theta_*self.nu_-0.5*self.sigma_**2*self.nu_)/self.nu_)*T
                phi = np.exp(1j*u*mu)*((1-1j*self.nu_*self.theta_*u+0.5*self.nu_*self.sigma_**2*u**2)**(-T/self.nu_))

            return phi

In [68]:
VG = VarianceGamma(sigmaVG,nu,thetaVG)

In [69]:
VG.testInit()

This is my VG


In [70]:
class Heston:

    V0_ = 0.0
    theta_ = 0.0
    sigma_ = 0.0
    kappa_ = 0.0
    rho_ = 0.0

    def __init__(self,V0, kappa,theta,sigma,rho):
        self.V0_ = V0
        self.theta_ = theta
        self.sigma_ = sigma
        self.kappa_ = kappa
        self.rho_ = rho

    def testInit(self):
        print("This is my Heston")

    def CharacteristicFunc(self,u,S0,T,r,q):

        temp = (self.kappa_-1j*self.rho_*self.sigma_*u)
        g = np.sqrt((self.sigma_**2)*(u**2+1j*u)+temp**2)
        
        pow1 = 2*self.kappa_*self.theta_/(self.sigma_**2)

        numer1 = (self.kappa_*self.theta_*T*temp)/(self.sigma_**2) + 1j*u*T*r + 1j*u*math.log(S0)
        log_denum1 = pow1 * np.log(np.cosh(g*T/2)+(temp/g)*np.sinh(g*T/2))
        temp2 = ((u*u+1j*u)*self.V0_)/(g/np.tanh(g*T/2)+temp)
        log_phi = numer1 - log_denum1 - temp2
        phi = np.exp(log_phi)
        return phi

In [71]:
Heston = Heston(V0,thetaH,sigmaH,kappa,rho)

In [72]:
Heston.testInit()

This is my Heston


In [73]:
class FFT:

    n_ = 0.0
    N_ = 0.0
    alpha_ = 0.0
    eta_ = 0.0

    def __init__(self,n,alpha,eta):
        self.n_ = n
        self.N_ = 2**self.n_
        self.alpha_ = alpha
        self.eta_ = eta

    def runFFT(self,model,S0,K,T,r,q):
        
         # increment in log strike space
        lda = (2*np.pi/self.N_)/self.eta_
    
        #beta = np.log(S0)-N*lda/2
        beta = np.log(K)
    
         # forming vector x and strikes km for m=1,...,N
        X = np.zeros((self.N_))
        km = np.zeros((self.N_))
         

        # discount factor
        df = math.exp(-r*T)
    
        nuJ = np.arange(self.N_)*self.eta_
        psi_nuJ = model.CharacteristicFunc(nuJ-(self.alpha_+1)*1j, S0, T,r,q)/((self.alpha_ + 1j*nuJ)*(self.alpha_+1+1j*nuJ))

        for j in range(self.N_):  
            km[j] = beta+j*lda
            if j == 0:
                wJ = (self.eta_/2)
            else:
                wJ = self.eta_
            X[j] = np.exp(-1j*beta*nuJ[j])*df*psi_nuJ[j]*wJ

        Y = np.fft.fft(X)
        cT_km = np.zeros((self.N_))  
        
        for i in range(self.N_):
            multiplier = np.exp(-self.alpha_*km[i])/math.pi
            cT_km[i] = multiplier*np.real(Y[i])
    
        return km, cT_km


In [90]:

#step size
eta = 0.25

#damper for the FFT
alpha = 1.5

n = 8\

fft = FFT(n,alpha,eta)

start_time = time.time()

km, cT_km = fft.runFFT(Heston,S0,K,T,r,q)

cT_k = cT_km[0]

elapsed_time = time.time() - start_time
    
#cT_k = np.interp(np.log(0.5), km, cT_km)
print('Option price is %0.6f' % cT_k)

#print("Option via FFT: for strike %s the option premium is %6.4f" % (np.exp(k), cT_km[0]))
print('FFT execution time was %0.7f' % elapsed_time)

Option price is 10.901860
FFT execution time was 0.0114522
